<a href="https://colab.research.google.com/github/Santosh-Gupta/SparseTorch/blob/master/TruePytorchWord2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy
import random 
import hashlib

import torch.optim as optim
from tqdm import tqdm
import sys
import tensorflow as tf
import collections
import numpy as np

from collections import deque
numpy.random.seed(12345)

use_cuda = True
# if use_cuda and torch.cuda.is_available():
#     net.cuda()

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import collections
import hashlib
import math
import os
import random
import sys
from tempfile import gettempdir
import zipfile

data_index = 0
vocabulary_size = 50000

In [0]:
import os
from tempfile import gettempdir
import urllib.request
from six.moves import xrange
import zipfile


In [0]:
def _hash_file(fpath):
    hasher = hashlib.sha256()
    with open(fpath, 'rb') as fpath_file:
        for chunk in iter(lambda: fpath_file.read(65535), b''):
            hasher.update(chunk)
    return hasher.hexdigest()

In [0]:
  # Create the directory for TensorBoard variables if there is not.
  if not os.path.exists('log_dir'):
    os.makedirs('log_dir')

  # Step 1: Download the data.
  # Note: Source website does not support HTTPS right now.
  url = 'http://mattmahoney.net/dc/'

  # pylint: disable=redefined-outer-name
  def maybe_download(filename, expected_bytes, sha256=None):
    """Download a file if not present, and make sure it's the right size."""
    local_filename = os.path.join(gettempdir(), filename)
    if not os.path.exists(local_filename):
      local_filename, _ = urllib.request.urlretrieve(url + filename,
                                                     local_filename)
    statinfo = os.stat(local_filename)

    if sha256 and _hash_file(local_filename) != sha256:
      raise Exception('Failed to verify ' + local_filename + ' due to hash '
                      'mismatch. Can you get to it with a browser?')

    if statinfo.st_size == expected_bytes:
      print('Found and verified', filename)
    else:
      print(statinfo.st_size)
      raise Exception('Failed to verify ' + local_filename +
                      '. Can you get to it with a browser?')
    return local_filename

In [5]:

filename = maybe_download(
    'text8.zip',
    31344016,
    sha256='a6640522afe85d1963ad56c05b0ede0a0c000dddc9671758a6cc09b7a38e5232')

Found and verified text8.zip


In [6]:
 # Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with zipfile.ZipFile(filename) as f:
      data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

vocabulary = read_data(filename)
print('Data size', len(vocabulary))

# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 50000

Data size 17005207


In [0]:
def build_dataset(words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = {}
    for word, _ in count:
      dictionary[word] = len(dictionary)
    data = []
    unk_count = 0
    for word in words:
      index = dictionary.get(word, 0)
      if index == 0:  # dictionary['UNK']
        unk_count += 1
      data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [8]:
  # Filling 4 global variables:
  # data - list of codes (integers from 0 to vocabulary_size-1).
  #   This is the original text but words are replaced by their codes
  # count - map of words(strings) to count of occurrences
  # dictionary - map of words(strings) to their codes(integers)
  # reverse_dictionary - maps codes(integers) to words(strings)
data, count, unused_dictionary, reverse_dictionary = build_dataset(
    vocabulary, vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


In [0]:
	  # Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window, negRate):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window

    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    negs = np.ndarray(shape=(batch_size, negRate), dtype=np.int32)

    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)  # pylint: disable=redefined-builtin
    if data_index + span > len(data):
      data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
      context_words = [w for w in range(span) if w != skip_window]
      words_to_use = random.sample(context_words, num_skips)
      for j, context_word in enumerate(words_to_use):
        batch[i * num_skips + j] = buffer[skip_window]
        labels[i * num_skips + j, 0] = buffer[context_word]
        excludez = np.concatenate([  labels[i * num_skips + j] , [batch[i * num_skips + j] ]])
        ij=0
        while ij < negRate:
            numCan = int(random.random() * (vocabulary_size/10))
            if numCan in excludez or numCan in negs[ i * num_skips + j ]:
                continue
            negs[i * num_skips + j, ij] = numCan
            ij = ij + 1

      if data_index == len(data):
        buffer.extend(data[0:span])
        data_index = span
      else:
        buffer.append(data[data_index])
        data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels, negs


In [10]:
batch, labels, negz = generate_batch(batch_size=8, num_skips=2, skip_window=1, negRate = 4)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0],
        reverse_dictionary[labels[i, 0]],
        '->', negz[i],  reverse_dictionary[negz[i][0]] )

3081 originated -> 12 as -> [2033 2579  505 2175] contract
3081 originated -> 5234 anarchism -> [3318 3818 1729  326] keys
12 as -> 3081 originated -> [4832 1771 2300 3135] fate
12 as -> 6 a -> [2471  744 4411 4372] husband
6 a -> 12 as -> [4487 1409 3892 2625] noise
6 a -> 195 term -> [4249   64 3954 1670] mineral
195 term -> 2 of -> [3594 4633 2489 4398] developments
195 term -> 6 a -> [ 194 4642 4625  655] international


In [0]:

class SkipGramModel(nn.Module):
    """Skip gram model of word2vec.
    Attributes:
        emb_size: Embedding size.
        emb_dimention: Embedding dimention, typically from 50 to 500.
        u_embedding: Embedding for center word.
        v_embedding: Embedding for neibor words.
    """

    def __init__(self, emb_size, emb_dimension):
        """Initialize model parameters.
        Apply for two embedding layers.
        Initialize layer weight
        Args:
            emb_size: Embedding size.
            emb_dimention: Embedding dimention, typically from 50 to 500.
        Returns:
            None
        """
        super(SkipGramModel, self).__init__()
        self.emb_size = emb_size
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=True)
        self.v_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=True)
        self.init_emb()

    def init_emb(self):
        """Initialize embedding weight like word2vec.
        The u_embedding is a uniform distribution in [-0.5/em_size, 0.5/emb_size], and the elements of v_embedding are zeroes.
        Returns:
            None
        """
        initrange = 0.5 / self.emb_dimension
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.normal_(mean=0, std=initrange)
        # self.v_embeddings.weight.data.uniform_(-0, 0)
        # self.v_embeddings.weight.data.uniform_(-initrange, initrange)

    def forward(self, pos_u, pos_v, neg_v):
        """Forward process.
        As pytorch designed, all variables must be batch format, so all input of this method is a list of word id.
        Args:
            pos_u: list of center word ids for positive word pairs.
            pos_v: list of neibor word ids for positive word pairs.
            neg_u: list of center word ids for negative word pairs.
            neg_v: list of neibor word ids for negative word pairs.
        Returns:
            Loss of this process, a pytorch variable.
        """
        emb_u = self.u_embeddings(pos_u)
        emb_v = self.v_embeddings(pos_v)
        score = torch.mul(emb_u, emb_v).squeeze()
        score = torch.sum(score, dim=1)
        score = F.logsigmoid(score)

        neg_emb_v = self.v_embeddings(neg_v)
        neg_score = torch.bmm(neg_emb_v, emb_u.unsqueeze(2)).squeeze()
        neg_score = F.logsigmoid(-1 * neg_score)
        return -1 * (torch.sum(score)+torch.sum(neg_score))


    def save_embedding(self, id2word, file_name, use_cuda):
        """Save all embeddings to file.
        As this class only record word id, so the map from id to word has to be transfered from outside.
        Args:
            id2word: map from word id to word.
            file_name: file name.
        Returns:
            None.
        """
        if use_cuda:
            embedding = self.u_embeddings.weight.cpu().data.numpy()
        else:
            embedding = self.u_embeddings.weight.data.numpy()
        fout = open(file_name, 'w')
        fout.write('%d %d\n' % (len(id2word), self.emb_dimension))
        for wid, w in id2word.items():
            e = embedding[wid]
            e = ' '.join(map(lambda x: str(x), e))
            fout.write('%s %s\n' % (w, e))


In [0]:
batch, labels, negz = generate_batch(batch_size=8, num_skips=4, skip_window=2, negRate= 4)


In [13]:
print(batch)

[3134 3134 3134 3134   46   46   46   46]


In [14]:
print(labels)

[[ 195]
 [  46]
 [   2]
 [  59]
 [3134]
 [ 156]
 [  59]
 [   2]]


In [0]:
batchTensor = torch.from_numpy(batch)

In [16]:
batchTensor

tensor([3134, 3134, 3134, 3134,   46,   46,   46,   46], dtype=torch.int32)

In [17]:
print(batchTensor)

tensor([3134, 3134, 3134, 3134,   46,   46,   46,   46], dtype=torch.int32)


In [18]:
print(batchTensor.long())

tensor([3134, 3134, 3134, 3134,   46,   46,   46,   46])


In [19]:
type(batchTensor)

torch.Tensor

In [20]:
print(labels)

[[ 195]
 [  46]
 [   2]
 [  59]
 [3134]
 [ 156]
 [  59]
 [   2]]


In [0]:
batch, labels, negz = generate_batch(batch_size=8, num_skips=2, skip_window=2, negRate= 4)
batchTensor = torch.from_numpy(batch)
labels = labels.flatten()
LabelTensor = torch.from_numpy(labels)
negTensor = torch.from_numpy(negz)

In [0]:
labels = labels.flatten()

In [0]:
LabelTensor = torch.from_numpy(labels)

In [24]:
print(LabelTensor)

tensor([3134,  156,   59,   46,  742,  156,  128,  477], dtype=torch.int32)


In [0]:
negTensor = torch.from_numpy(negz)

In [26]:
print(negTensor)

tensor([[4496,  168, 3551, 2152],
        [ 502, 3338,  512, 4771],
        [2065, 4546, 2119, 1391],
        [4476, 4895, 4344, 2149],
        [1253, 4600, 3448,  892],
        [ 863, 4854, 2477, 1379],
        [4370, 3351, 3106, 4225],
        [2331, 2268, 1440, 4517]], dtype=torch.int32)


In [27]:
output_file_name = 'theOutpule.file'
emb_size = 50000
print('embed size is ', emb_size)
emb_dimension = 128
batch_size = 128
window_size = 4
iterationsMax = 40000 #200001
initial_lr = 1.0
skip_gram_model = SkipGramModel(emb_size, emb_dimension)
print('got skip gram model')
use_cuda = torch.cuda.is_available()
print('use cuda ? ', use_cuda)
if use_cuda:
    skip_gram_model.cuda()
print('mounted skip gram to cude')
optimizer = optim.SGD(
    skip_gram_model.parameters(), lr=initial_lr)
print('got optimizer')

embed size is  50000
got skip gram model
use cuda ?  True
mounted skip gram to cude
got optimizer


In [0]:
initial_lr = 1.0
optimizer = optim.SGD(
    skip_gram_model.parameters(), lr=0.001)

In [0]:
optimizer = optim.Adagrad(
    skip_gram_model.parameters())

In [30]:
runningLoss = 0

batch_size = 128


for i in range(1000001):
    # print('does it even go here', i)
    # pos_pairs = dataz.get_batch_pairs(batch_size,
    #                                         window_size)
    # print('pos pairs is ', pos_pairs)
    # neg_v = dataz.get_neg_v_neg_sampling(pos_pairs, 5)
    # print('negs are ', neg_v)
    # pos_u = [pair[0] for pair in pos_pairs]
    # print('pos_u is ', pos_u)
    # pos_v = [pair[1] for pair in pos_pairs]
    # print('pos_v is ', pos_v)

# skip_window = 1  # How many words to consider left and right.
# num_skips = 2  # How many times to reuse an input to generate a label.

    batch, labels, negz = generate_batch(batch_size=batch_size, num_skips=2, skip_window=1, negRate= 64)
    batchTensor = torch.from_numpy(batch)
    labels = labels.flatten()
    LabelTensor = torch.from_numpy(labels)
    negTensor = torch.from_numpy(negz)

    pos_u = Variable(torch.LongTensor(batchTensor.long()))
    pos_v = Variable(torch.LongTensor(LabelTensor.long()))
    neg_v = Variable(torch.LongTensor(negTensor.long()))

    if use_cuda:
        pos_u = pos_u.cuda()
        pos_v = pos_v.cuda()
        neg_v = neg_v.cuda()

    # optimizer.zero_grad() #set gradients to zero, need to do this at every step or gradients accumulate
    optimizer.zero_grad()

    loss = skip_gram_model.forward(pos_u, pos_v, neg_v)
    runningLoss = runningLoss + loss.data.item()
    # if i%100 == 0:
    #     print('loss is ', loss.data)

    loss.backward()
    optimizer.step()


    if i%500 == 0:
        print('i is ', i)
        print('loss is ', runningLoss/500)
        runningLoss = 0

    # process_bar.set_description("Loss: %0.8f, lr: %0.6f" %
    #                             (loss.data[0],
    #                                 self.optimizer.param_groups[0]['lr']))
    # if i * batch_size % 100000 == 0:
    #     lr = self.initial_lr * (1.0 - 1.0 * i / batch_count)
    #     for param_group in self.optimizer.param_groups:
    #         param_group['lr'] = lr

# self.skip_gram_model.save_embedding(
#     self.data.id2word, self.output_file_name, self.use_cuda)

i is  0
loss is  11.533966796875
i is  500
loss is  2781.2825708007813
i is  1000
loss is  1745.5502824707032
i is  1500
loss is  1458.7799700927735
i is  2000
loss is  1298.6775208740235
i is  2500
loss is  1199.1390642700196
i is  3000
loss is  1106.2075556640625
i is  3500
loss is  996.426608947754
i is  4000
loss is  981.5178012695312
i is  4500
loss is  904.9059776611329
i is  5000
loss is  838.0480902709961
i is  5500
loss is  895.2369212646485
i is  6000
loss is  881.9654464111328
i is  6500
loss is  784.1718362426758
i is  7000
loss is  812.1385935058594
i is  7500
loss is  740.9788065185547
i is  8000
loss is  792.13784375
i is  8500
loss is  735.7298042602539
i is  9000
loss is  703.5626795654297
i is  9500
loss is  707.3482135620117
i is  10000
loss is  670.7172040710449
i is  10500
loss is  695.1228662109374
i is  11000
loss is  689.4463616333007
i is  11500
loss is  636.4275872802734
i is  12000
loss is  640.1980825195312
i is  12500
loss is  654.956731628418
i is  13000
l

KeyboardInterrupt: ignored

In [0]:
print(loss.data.item())

In [0]:
# norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
#     normalized_embeddings = embeddings / norm
#     valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings,
#                                               valid_dataset)
#     similarity = tf.matmul(
#         valid_embeddings, normalized_embeddings, transpose_b=True)

In [0]:
squardd = torch.pow(skip_gram_model.u_embeddings.weight.data, 2)

In [0]:
print(squardd)

In [0]:
sim = torch.mul(skip_gram_model.u_embeddings.weight.data, skip_gram_model.v_embeddings.weight.data)

In [0]:
embedNorm = torch.norm(skip_gram_model.u_embeddings.weight.data, p=2)

In [32]:
print(embedNorm)

tensor(96.4272, device='cuda:0')


In [0]:
normalizedWeights = torch.div(skip_gram_model.u_embeddings.weight.data, embedNorm)

In [0]:
validexamples = Variable(torch.LongTensor([ 4, 23, 45, 28, 29, 12, 11, 16, 20, 24, 28, 32, 34, 82, 94, 39, 54, 21, 43, 47, 52, 56, 60, 66, 73, 77, 82, 88, 99, 97])) 

In [0]:
 validExampleTensors = skip_gram_model.u_embeddings( validexamples.cuda() )

In [0]:
normalizedValid = torch.div(validExampleTensors, embedNorm)

In [0]:
finalSim = torch.mm(normalizedValid, normalizedWeights.t() )

In [46]:
print(finalSim)

tensor([[ 7.1871e-05,  8.0662e-05,  3.2188e-05,  ..., -1.1274e-07,
          2.2949e-07, -3.3377e-07],
        [ 6.0831e-05,  2.7576e-06,  4.2986e-05,  ..., -4.0243e-08,
          1.1123e-07, -5.2457e-07],
        [ 5.6719e-05,  1.7365e-04,  7.0849e-05,  ...,  4.2774e-08,
         -2.5164e-07,  1.5459e-07],
        ...,
        [ 5.4501e-05,  1.3877e-05,  6.2630e-05,  ...,  3.5167e-08,
         -2.1699e-07, -1.1320e-08],
        [ 5.8752e-05,  5.6476e-05,  1.0551e-04,  ..., -3.3040e-09,
         -2.6968e-07, -2.6026e-08],
        [ 4.1739e-05,  1.4736e-05,  4.6135e-05,  ...,  1.5004e-07,
         -1.7628e-07, -3.1063e-08]], device='cuda:0', grad_fn=<MmBackward>)


In [47]:
print(finalSim[1].size())

torch.Size([50000])


In [0]:
finalSim.size()

In [0]:
simNumpy = finalSim.cpu().detach().numpy() 

In [49]:
print(simNumpy)

[[ 7.18711599e-05  8.06619064e-05  3.21883235e-05 ... -1.12742200e-07
   2.29486503e-07 -3.33770345e-07]
 [ 6.08308364e-05  2.75757816e-06  4.29855609e-05 ... -4.02430906e-08
   1.11230314e-07 -5.24570225e-07]
 [ 5.67194002e-05  1.73650129e-04  7.08488369e-05 ...  4.27737383e-08
  -2.51640017e-07  1.54590197e-07]
 ...
 [ 5.45012663e-05  1.38767919e-05  6.26304100e-05 ...  3.51672966e-08
  -2.16990472e-07 -1.13196634e-08]
 [ 5.87524701e-05  5.64756629e-05  1.05511950e-04 ... -3.30402661e-09
  -2.69675041e-07 -2.60259139e-08]
 [ 4.17387564e-05  1.47359297e-05  4.61345116e-05 ...  1.50041501e-07
  -1.76283464e-07 -3.10632586e-08]]


In [50]:
simNumpy[1].shape

(50000,)

In [51]:
valid_size = [ 4, 23, 45, 28, 29, 12, 11, 16, 20, 24, 28, 32, 34, 82, 94, 39, 54, 21, 43, 47, 52, 56, 60, 66, 73, 77, 82, 88, 99, 97]

for i in range(len(valid_size)):
    valid_word = reverse_dictionary[valid_size[i]]
    top_k = 80  # number of nearest neighbors
    nearest = (-simNumpy[i, :]).argsort()[1:top_k + 1]
    log_str = 'Nearest to %s:' % valid_word
    for k in xrange(top_k):
        close_word = reverse_dictionary[nearest[k]]
        log_str = '%s %s,' % (log_str, close_word)
    print(log_str)

Nearest to one: eight, seven, one, zero, six, five, four, three, two, th, century, years, anarchism, km, isbn, c, december, anarchist, million, october, april, m, january, b, anarchists, february, november, september, august, july, march, autism, s, lincoln, late, rand, june, d, f, bc, births, n, p, about, g, feet, autistic, e, deaths, aristotle, x, mid, apollo, pp, miles, female, mi, male, kings, billion, taggart, approximately, links, external, addition, h, early, least, est, ft, days, u, months, l, rest, achilles, lack, year, rearden, ayn,
Nearest to seven: zero, eight, seven, six, five, four, three, one, two, th, century, years, anarchism, km, isbn, c, million, anarchist, december, m, b, october, april, january, anarchists, february, november, august, september, july, march, autism, d, lincoln, f, late, rand, s, june, bc, births, about, n, p, e, feet, g, x, deaths, autistic, aristotle, pp, mid, miles, apollo, female, male, mi, billion, external, u, approximately, kings, links, tagg

In [0]:
valid_size = [ 4, 23, 45, 283, 292, 12, 113, 167, 200, 204, 208, 432, 354, 382, 194, 339, 54, 221, 121, 111]

for i in valid_size:
    valid_word = reverse_dictionary[i]
    top_k = 8  # number of nearest neighbors
    nearest = (-simNumpy[i, :]).argsort()[1:top_k + 1]
    log_str = 'Nearest to %s:' % valid_word
    for k in xrange(top_k):
        close_word = reverse_dictionary[nearest[k]]
        log_str = '%s %s,' % (log_str, close_word)
    print(log_str)
for i in valid_size:
    valid_word = reverse_dictionary[i]
    top_k = 8  # number of nearest neighbors
    nearest = (-simNumpy[i, :]).argsort()[1:top_k + 1]
    log_str = 'Nearest to %s:' % valid_word
    for k in xrange(top_k):
        close_word = reverse_dictionary[nearest[k]]
        log_str = '%s %s,' % (log_str, close_word)
    print(log_str)